In [1]:
import mdtraj
import glob

import numpy as np
import alphaspace2 as AS2
import alphaspace2.Features as AS2_feat

from scipy.spatial.distance import cdist
from scipy.cluster.hierarchy import fcluster, linkage
from alphaspace2.functions import _binCluster, _group
from alphaspace2.Cluster import _DPocket
from collections import defaultdict 

%load_ext autoreload
%autoreload 2

In [ ]:
def strip_h(input_file,output_file):
    '''
    input_file and output_file need to be in pdb or pdbqt format 
    '''
    inputlines = open(input_file,'r').readlines()
    no_h_lines = [l for l in inputlines if not l.split()[-1].startswith('H')] 
    output = open(output_file,'w')
    output.writelines(no_h_lines)
    output.close()
    
# strip_h("1M17_A.pdb", "1M17_A1.pdb")
# strip_h("1M17_A1.pdbqt", "1M17_A1_noH.pdbqt")

In [5]:
## pdb2pqr corrected PDB file with hydrogens removed
prot = mdtraj.load('../data/1M17_data/PDB_files/1M17_A1.pdb')

## ligand PDB file
lig = mdtraj.load('../data/1M17_data/PDB_files/1M17_A_AQ4.pdb')

## PDBQT file (generated from the pdb2pqr file) with hydrogens removed
AS2.annotateVinaAtomTypes(pdbqt="../data/1M17_data/PDB_files/1M17_A1_noH.pdbqt", receptor=prot)
ss_prot = AS2.Snapshot()
ss_prot.run(prot, lig, score_function="Lin_F9")


In [6]:
protease_contact_pockets, protease_contact_pockets_alphas, contact_LinF9_bscore_dict, contact_space_dict,\
    protease_contact_betas, contact_occupancy_dict = defaultdict(dict), defaultdict(dict), defaultdict(dict),\
    defaultdict(dict), defaultdict(dict), defaultdict(dict)

for px,pocket in enumerate(ss_prot.pockets):
    if pocket.isContact:
        protease_contact_betas[px] = pocket
        protease_contact_pockets[px] = np.array([b.xyz for b in pocket.betas])
        protease_contact_pockets_alphas[px] = np.array([a.xyz for a in pocket.alphas])
        contact_LinF9_bscore_dict[px] = np.sum(np.array([min(b.scores) for b in pocket.betas]))
        contact_space_dict[px] = np.sum(np.array([b.space for b in pocket.betas]))
        contact_occupancy_dict[px] = pocket.occupancy

In [7]:
ss_prot.run(prot, lig, score_function="Vina")

contact_Vina_bscore_dict = defaultdict(dict)
for px,pocket in enumerate(ss_prot.pockets):
    if pocket.isContact:
        contact_Vina_bscore_dict[px] = np.sum(np.array([min(b.scores) for b in pocket.betas]))


In [8]:
sort_key = sorted(contact_space_dict.items(), key=lambda x:x[1], reverse=True)

print("Rank, % Occupancy, Space, β-score (Lin F9), β-score (Vina)")
for i,pid in enumerate([item[0] for item in sort_key]): 
    print(f"{i}, {int(contact_occupancy_dict[pid]*100)}, {int(contact_space_dict[pid])}, {contact_LinF9_bscore_dict[pid]:2.1f}, {contact_Vina_bscore_dict[pid]:2.1f}")
    

Rank, % Occupancy, Space, β-score (Lin F9), β-score (Vina)
0, 80, 403, -5.8, -6.8
1, 61, 204, -2.1, -2.4
2, 98, 163, -2.3, -3.4
3, 61, 68, -1.4, -1.6
